# Model BertForSequenceClassification na danych z 4 etykietami

In [16]:
# Importy
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE


In [2]:
df = pd.read_csv("reviews_eda1b.csv")

In [3]:
df.columns

Index(['Link', 'Autor', 'Tytuł', 'Data_recenzji', 'Treść_recenzji',
       'Średnia_ocen', 'Liczba_ocen', 'Liczba_czytelników', 'Rok_recenzji',
       'sentyment'],
      dtype='object')

In [4]:
# Moje dane
texts = df["Treść_recenzji"].tolist()  # Lista tekstów
labels = df["sentyment"].tolist()  # Lista etykiet sentymentu

In [ ]:
texts

base

In [6]:
# Dzielę na zbiór treningowy, walidacyjny i testowy
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)

In [7]:
# Tokenizacja tekstu
tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-multilingual-cased')

tokenized_train = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
tokenized_val = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")
tokenized_test = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")

In [8]:
# Konwersja etykiet do tensorów
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

In [9]:
# Muszę zrobić TensorDataset
train_dataset = TensorDataset(tokenized_train.input_ids, tokenized_train.attention_mask, tokenized_train.token_type_ids, train_labels)
val_dataset = TensorDataset(tokenized_val.input_ids, tokenized_val.attention_mask, tokenized_val.token_type_ids, val_labels)
test_dataset = TensorDataset(tokenized_test.input_ids, tokenized_test.attention_mask, tokenized_test.token_type_ids, test_labels)

In [10]:
#jaikes błedy, więc robię  DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [11]:
# Inicjalizuję model
model = BertForSequenceClassification.from_pretrained('google-bert/bert-base-multilingual-cased', num_labels=4)
optimizer = AdamW(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at google-bert/bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-ber

In [13]:
# Trenowanie modelu
num_epochs = 3

for epoch in range(num_epochs):
    print(epoch)
    model.train()
    for batch in train_dataloader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'token_type_ids': batch[2], 'labels': batch[3]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

0
1
2


In [14]:
# Ewaluacja modelu
model.eval()
all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'token_type_ids': batch[2]}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predictions.tolist())
        all_true_labels.extend(batch[3].tolist())


In [15]:
# wyniki
print(classification_report(all_true_labels, all_predictions))
print("Accuracy:", accuracy_score(all_true_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        36
           1       0.00      0.00      0.00       184
           2       0.63      1.00      0.77       750
           3       0.00      0.00      0.00       223

    accuracy                           0.63      1193
   macro avg       0.16      0.25      0.19      1193
weighted avg       0.40      0.63      0.49      1193

Accuracy: 0.6286672254819782


c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

smote

In [27]:
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
import torch
from sklearn.metrics import classification_report, accuracy_score

In [28]:
# Dziele na zbiór treningowy, walidacyjny i testowy
train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.1, random_state=42)

In [29]:
# Tokenizacja tekstu
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

tokenized_train = tokenizer(train_texts, padding=True, truncation=True, return_tensors="pt")
tokenized_val = tokenizer(val_texts, padding=True, truncation=True, return_tensors="pt")
tokenized_test = tokenizer(test_texts, padding=True, truncation=True, return_tensors="pt")


In [31]:
# Oversampling SMOTE
# były błedy, więc muszę przekształcic dane treningowe do postaci 2D array
X_train_smote = torch.cat([tokenized_train.input_ids, tokenized_train.attention_mask, tokenized_train.token_type_ids], dim=1).numpy()
smote = SMOTE(random_state=42, sampling_strategy='auto')
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_smote, train_labels.numpy())


In [32]:
# Dzielę dane z powrotem na tokeny i etykiety
tokenized_train_resampled = torch.tensor(X_train_resampled[:, :tokenized_train.input_ids.shape[1]]), \
                            torch.tensor(X_train_resampled[:, tokenized_train.input_ids.shape[1]:2*tokenized_train.input_ids.shape[1]]), \
                            torch.tensor(X_train_resampled[:, 2*tokenized_train.input_ids.shape[1]:]), \
                            torch.tensor(y_train_resampled)


In [33]:
# Tworzę DataLoader dla danych po oversamplingu
train_dataloader_resampled = DataLoader(TensorDataset(*tokenized_train_resampled), batch_size=8, shuffle=True)

In [35]:
# Oversampling SMOTE dla danych walidacyjnych 
X_val_smote = torch.cat([tokenized_val.input_ids, tokenized_val.attention_mask, tokenized_val.token_type_ids], dim=1).numpy()
X_val_resampled, y_val_resampled = smote.fit_resample(X_val_smote, val_labels.numpy())

tokenized_val_resampled = torch.tensor(X_val_resampled[:, :tokenized_val.input_ids.shape[1]]), \
                          torch.tensor(X_val_resampled[:, tokenized_val.input_ids.shape[1]:2*tokenized_val.input_ids.shape[1]]), \
                          torch.tensor(X_val_resampled[:, 2*tokenized_val.input_ids.shape[1]:]), \
                          torch.tensor(y_val_resampled)

val_dataloader_resampled = DataLoader(TensorDataset(*tokenized_val_resampled), batch_size=8, shuffle=False)

# Oversampling SMOTE dla danych testowych 
X_test_smote = torch.cat([tokenized_test.input_ids, tokenized_test.attention_mask, tokenized_test.token_type_ids], dim=1).numpy()
X_test_resampled, y_test_resampled = smote.fit_resample(X_test_smote, test_labels.numpy())

tokenized_test_resampled = torch.tensor(X_test_resampled[:, :tokenized_test.input_ids.shape[1]]), \
                           torch.tensor(X_test_resampled[:, tokenized_test.input_ids.shape[1]:2*tokenized_test.input_ids.shape[1]]), \
                           torch.tensor(X_test_resampled[:, 2*tokenized_test.input_ids.shape[1]:]), \
                           torch.tensor(y_test_resampled)

test_dataloader_resampled = DataLoader(TensorDataset(*tokenized_test_resampled), batch_size=8, shuffle=False)

In [34]:
# Konwersja etykiet do tensorów
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
test_labels = torch.tensor(test_labels)

C:\Users\konta\AppData\Local\Temp\ipykernel_22544\4000818539.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels)
C:\Users\konta\AppData\Local\Temp\ipykernel_22544\4000818539.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  val_labels = torch.tensor(val_labels)
C:\Users\konta\AppData\Local\Temp\ipykernel_22544\4000818539.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_labels = torch.tensor(test_labels)


In [40]:
from torch.optim import Adam

In [41]:
# Inicjalizuję modelu
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)
optimizer = Adam(model.parameters(), lr=5e-5)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [42]:
# Trening modelu
num_epochs = 3

for epoch in range(num_epochs):
    print(epoch)
    model.train()
    for batch in train_dataloader_resampled:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'token_type_ids': batch[2], 'labels': batch[3]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

0
1
2


In [43]:
# Ewaluacja modelu
model.eval()
all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in val_dataloader_resampled:
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'token_type_ids': batch[2]}
        outputs = model(**inputs)
        predictions = torch.argmax(outputs.logits, dim=1)
        all_predictions.extend(predictions.tolist())
        all_true_labels.extend(batch[3].tolist())

In [44]:
# Wyniki
print(classification_report(all_true_labels, all_predictions))
print("Accuracy:", accuracy_score(all_true_labels, all_predictions))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40       300
           1       0.00      0.00      0.00       300
           2       0.00      0.00      0.00       300
           3       0.00      0.00      0.00       300

    accuracy                           0.25      1200
   macro avg       0.06      0.25      0.10      1200
weighted avg       0.06      0.25      0.10      1200

Accuracy: 0.25


c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\konta\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,